In [1]:
from qiskit.primitives import Estimator
from qiskit.circuit.library import ExcitationPreserving
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import COBYLA

from circuit_knitting.cutting.gate_and_wire_cutting.algorithms.cut_vqe import CutVQE

In [2]:
# Use a list of H2 observables from the PySCF driver
H2_op_list = [('II', -1.0523732457728596),
             ('IZ', 0.39793742484317934),
             ('XX', 0.18093119978423144),
             ('ZI', -0.39793742484317934),
             ('ZZ', -0.011280104256235268)]

# Problem specification. Hamiltonian for H2 at 0.735A interatomic distance.
H2_op = SparsePauliOp.from_list(H2_op_list)

In [3]:
num_qubits = 2
circuit = ExcitationPreserving(num_qubits, flatten=True, reps=2, skip_final_rotation_layer=True).decompose(reps=2)

In [4]:
# See how many parameters are in the ansatz
num_params = circuit.num_parameters
print(f'Num parameters: {num_params}')

# List the observables
observables = H2_op.paulis
print(f'Observables: {observables}')

Num parameters: 6
Observables: ['II', 'IZ', 'XX', 'ZI', 'ZZ']


In [5]:
# Create a callback function that will be used to get VQE data during the run 
def callback(eval_count, parameters, mean, std):
    print(f"Round num: {eval_count}, energy: {mean}, parameters: {parameters}")

In [ ]:
# Use CutVQE to cut the circuit and run VQE
estimator = Estimator()
optimizer = COBYLA(maxiter=80)
vqe = CutVQE(estimator, circuit, optimizer, observables=observables, shots=2**12, max_subcircuit_width=1, max_cuts=10, num_subcircuits=[2], model='gurobi', num_samples=1500, callback=callback)
result = vqe.compute_minimum_eigenvalue(H2_op)
print(result)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-10
Set parameter TimeLimit to value 300
Set parameter Cutoff to value 1e+100
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.0.0 23A344)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 213 rows, 91 columns and 718 nonzeros
Model fingerprint: 0x965564a0
Variable types: 0 continuous, 91 integer (80 binary)
Coefficient statistics:
  Matrix range     [1e-02, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 8.0000000
Presolve removed 213 rows and 91 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 8 

Optimal solution found 

In [6]:
# Compare against results obtained without cutting
estimator = Estimator()
optimizer = COBYLA(maxiter=80)
no_cut_vqe = VQE(ansatz=circuit, optimizer=optimizer, estimator=estimator, callback=callback)
no_cut_result = no_cut_vqe.compute_minimum_eigenvalue(H2_op)
print(no_cut_result)

Round num: 1, energy: -1.063653350029094, parameters: [5.266625166980573, 0.49791650269713106, 2.439510071705371, -0.2680065871872719, -1.6805244099690828, -5.772360907933034]
Round num: 2, energy: -1.0636533500290937, parameters: [6.266625166980573, 0.49791650269713106, 2.439510071705371, -0.2680065871872719, -1.6805244099690828, -5.772360907933034]
Round num: 3, energy: -1.0636533500290937, parameters: [5.266625166980573, 1.497916502697131, 2.439510071705371, -0.2680065871872719, -1.6805244099690828, -5.772360907933034]
Round num: 4, energy: -1.0636533500290937, parameters: [5.266625166980573, 0.49791650269713106, 3.439510071705371, -0.2680065871872719, -1.6805244099690828, -5.772360907933034]
Round num: 5, energy: -1.0636533500290937, parameters: [5.266625166980573, 0.49791650269713106, 2.439510071705371, 0.7319934128127281, -1.6805244099690828, -5.772360907933034]
Round num: 6, energy: -1.0636533500290937, parameters: [5.266625166980573, 0.49791650269713106, 2.439510071705371, -0.2